In [ ]:
import cv2, json, os
import matplotlib.pyplot as plt
import numpy as np
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from PIL import Image

In [ ]:
%run Common.ipynb

### VIA

Review labels created with VIA 2.0+...

In [ ]:
images = '<path to images e.g. C:\images>'
labels = '<path to labels e.g C:\labels.json>'
#labels = 'via_region_data.csv'

In [ ]:
normalized_regions = {}

with open(labels, 'r') as json_file:
    data = json.load(json_file)
    region_label = data['_via_settings']['ui']['image']['region_label']
    for row in data['_via_img_metadata']:
        filename = data['_via_img_metadata'][row]['filename']
        print(filename)
        regions = data['_via_img_metadata'][row]['regions']
        regions_count = len(regions)
        print(regions_count, 'regions')
        image = Image.open(os.path.join(images, filename))
        image = np.array(image)
        y, x, _ = image.shape
        region_labels = {}
        for region in regions:
            # Plot polyline...
            all_points_x = region['shape_attributes']['all_points_x']
            all_points_y = region['shape_attributes']['all_points_y']            
            pts = [list(pair) for pair in zip(all_points_x, all_points_y)]
            pts_array = np.array(pts)
            pts_array = pts_array.reshape(-1, 1, 2)
            cv2.polylines(image, pts_array, True, (0, 255, 255))
            # Compute equivalent bounding box...
            max_x = max(all_points_x)
            min_x = min(all_points_x)
            max_y = max(all_points_y)
            min_y = min(all_points_y)
            # Plot bounding box...
            roi = image[min_y:max_y, min_x:max_x]
            plt.imshow(roi)
            plt.show()
            # Prepare label for CustomVision.ai
            print(region['region_attributes'][region_label])
            region_labels[region['region_attributes'][region_label]] = [float(min_x / x), float(min_y / y), float((max_x - min_x) / x), float((max_y - min_y) / y)]
        plt.imshow(image)
        plt.show()
        normalized_regions[filename] = region_labels

## CustomVision.ai

Happy with the labels? Then, upload...

What is the CustomVision.ai project name?

In [ ]:
project_name = '<insert project name>'

In [ ]:
ENDPOINT = 'https://<insert region here>.api.cognitive.microsoft.com'

trainer = CustomVisionTrainingClient(custom_vision_training_key, endpoint=ENDPOINT)

In [ ]:
projects = trainer.get_projects()
project_names = [project.name for project in projects]

project = None

if not project_name in project_names:
    print('Project {0} not found. Creating...'.format(project_name))
    object_detection_domain_id = next(domain.id for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")
    project = trainer.create_project(name=project_name, domain_id=object_detection_domain_id)
    print('Created project {}.'.format(project_name))
else:
    project = [project for project in projects if project.name == project_name][0]

In [ ]:
project.name

Create tags...

In [ ]:
project_tags = trainer.get_tags(project.id)
project_tag_names = [project_tag.name for project_tag in project_tags]

tags = {}

with open(labels, 'r') as json_file:
    data = json.load(json_file)
    region_label = data['_via_settings']['ui']['image']['region_label']
    
    for option in data['_via_attributes']['region'][region_label]['options']:
        if not option in project_tag_names:
            print('Creating tag {0}...'.format(option))
            tags[option] = trainer.create_tag(project.id, option)
        else:
            print('Fetching tag {0}...'.format(option))
            project_tag = [project_tag for project_tag in project_tags if project_tag.name == option][0]
            tags[option] = project_tag

Upload to CustomVision.ai...

In [ ]:
for file_name in normalized_regions.keys():
    tagged_images_with_regions = []
    print('Uploading {0}...'.format(file_name))
    regions = []
    for normalized_region in normalized_regions[file_name]:
        x, y, w, h = normalized_regions[file_name][normalized_region]
        tag_id = tags[normalized_region].id
        regions.append(Region(tag_id=tag_id, left=x, top=y, width=w, height=h) )

    with open(os.path.join(images, file_name), mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

    upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)

    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)